# Notebook 6.2: Baichuan-13B

## 6.2.1 Overview

This is an example shows how to run [Baichuan-13B](https://github.com/baichuan-inc/Baichuan-13B) Chinese inference on low-cost PCs (without the need of discrete GPU) using [BigDL-LLM](https://github.com/intel-analytics/BigDL/tree/main/python/llm) APIs. Baichuan-13B is an open-source, commercially available large-scale language model developed by Baichuan Intelligent Technology following [Baichuan-7B](https://github.com/baichuan-inc/baichuan-7B). Baichuan-13B also can be found in [Huggingface models](https://huggingface.co/models) in following [link](https://huggingface.co/baichuan-inc/Baichuan-13B-Chat).

## 6.2.2 Installation

Install BigDL-LLM through:

In [ ]:
!pip install bigdl-llm[all]

The all option is for installing other required packages by BigDL-LLM.

## 6.2.3 Load Model and Tokenizer

### 6.2.3.1 Load Model

Load model with low-precision optimization(INT4) for lower resource cost using BigDL-LLM APIs, which convert the relevant layers in the model into INT4 format. 

> **Note**
>
> You can specify the argument `model_path` with both Huggingface repo id or local model path.

In [ ]:
from bigdl.llm.transformers import AutoModelForCausalLM

model_path = "baichuan-inc/Baichuan-13B-Chat"
model = AutoModelForCausalLM.from_pretrained(model_path,
                                             load_in_4bit=True,
                                             trust_remote_code=True)

### 6.2.3.2 Load Tokenizer

The quantized model is compatible with the tokenizer provided by the [Huggingface transformers library](https://huggingface.co/docs/transformers/index). Thus, you can directly use Huggingface transformers APIs to load tokenizer. A tokenizer maps between texts and lists of integers. We use it to encode input texts to integers for model to calculate, and decode the model output integers to texts for human to read.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path,
                                          trust_remote_code=True)

## 6.2.4 Save and Reload

There is a conversion from high-precision model to low-precision model in function `from_pretrained`, which is a time-cosuming process, especially for larger model. Therefore, we provide save/load APIs for low-precision model to avoid repeating this conversion process.

In [ ]:
# Save pretrained model
save_dir = "./baichuan/"
model.save_low_bit(save_dir)

# Load pretrained model again
model = AutoModelForCausalLM.load_low_bit(save_dir, trust_remote_code=True)

## 6.2.5 Inference

### 6.2.5.1 Create Prompt Template

Before generating, you need to create a prompt template. Here we give an example prompt template. You can tune the prompt based on your own model as well.

In [1]:
BAICHUAN_PROMPT_FORMAT = "<human>{prompt} <bot>"

### 6.2.5.2 Generate

Then, you can generate output with loaded model and tokenizer.

> **Note**
>
> `max_new_tokens` parameter in the `generate` function defines the maximum number of tokens to predict.

In [4]:
import time
import torch

prompt = "AI是什么？"
n_predict = 128
with torch.inference_mode():
        prompt = BAICHUAN_PROMPT_FORMAT.format(prompt=prompt)
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        st = time.time()
        # if your selected model is capable of utilizing previous key/value attentions
        # to enhance decoding speed, but has `"use_cache": false` in its model config,
        # it is important to set `use_cache=True` explicitly in the `generate` function
        # to obtain optimal performance with BigDL-LLM INT4 optimizations
        output = model.generate(input_ids,
                                max_new_tokens=n_predict)
        end = time.time()
        output_str = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f'Inference time: {end-st} s')
        print('-'*20, 'Output', '-'*20)
        print(output_str)

Inference time: 8.000465869903564 s
-------------------- Prompt --------------------
<human>AI是什么？ <bot>
-------------------- Output --------------------
<human>AI是什么？ <bot>人工智能(Artificial Intelligence，简称AI)是指由人制造出来的系统所表现出来的智能，通常是通过计算机系统实现的。这种智能来源于计算机程序和数据处理能力，可以模拟、扩展和辅助人类的认知功能。
